In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! pip install kaggle 
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download kartik2112/fraud-detection
! unzip /content/fraud-detection.zip

 93% 188M/202M [00:03<00:00, 64.8MB/s]
100% 202M/202M [00:03<00:00, 55.0MB/s]
Archive:  /content/fraud-detection.zip
  inflating: fraudTest.csv           
  inflating: fraudTrain.csv          


In [ ]:
import pandas as pd
import numpy as np
from tqdm import *
test_file = "/content/fraudTest.csv"
train_file = "/content/fraudTrain.csv"
usecols =['trans_date_trans_time','cc_num','merchant','category','amt','gender','city','state','zip','lat','long','city_pop','job','dob','merch_lat','merch_long','is_fraud']
state = ['TX']
pd.set_option("display.max_columns", None)

In [ ]:
train_df = pd.read_csv(train_file,usecols=usecols)

In [ ]:
df1 = train_df.loc[train_df['state'].isin(state)]

In [ ]:
merchant = list(set(df1['merchant']))
columns = ['id','type','category','mean_amt','min_amt','1Q_amt','2Q_amt','3Q_amt','max_amt']
merchantDF = pd.DataFrame(columns=columns)
for i in range(len(merchant)):
    mdf = df1.loc[df1['merchant']==merchant[i]]
    values_to_add = {}
    values_to_add['id'] = merchant[i]
    values_to_add['type'] = 'merchant'
    values_to_add['category'] = mdf['category'].value_counts().keys()[0]
    values_to_add['mean_amt'] = mdf['amt'].mean()
    values_to_add['1Q_amt'] = mdf['amt'].quantile(0.25)
    values_to_add['2Q_amt'] = mdf['amt'].quantile(0.5)
    values_to_add['3Q_amt'] = mdf['amt'].quantile(0.75)
    values_to_add['min_amt'] = mdf['amt'].min()
    values_to_add['max_amt'] = mdf['amt'].max()
    row_to_add = pd.Series(values_to_add, name=i)
    merchantDF = merchantDF.append(row_to_add)
    
        
merchantDF.to_csv('/content/drive/MyDrive/merchant_features.csv')

In [ ]:
users = list(set(df1['cc_num']))
columns = ['id','type','mean_amt','min_amt','1Q_amt','2Q_amt','3Q_amt','max_amt','gender','city','state','zip','lat','long','city_pop','job','age']
usersDF = pd.DataFrame(columns=columns)
for i in range(len(users)):
    udf = df1.loc[df1['cc_num']==users[i]]
    values_to_add = {}
    values_to_add['id'] = users[i]
    values_to_add['type'] = 'user'
    values_to_add['mean_amt'] = udf['amt'].mean()
    values_to_add['1Q_amt'] = udf['amt'].quantile(0.25)
    values_to_add['2Q_amt'] = udf['amt'].quantile(0.5)
    values_to_add['3Q_amt'] = udf['amt'].quantile(0.75)
    values_to_add['min_amt'] = udf['amt'].min()
    values_to_add['max_amt'] = udf['amt'].max()
    values_to_add['gender'] = udf['gender'].value_counts().keys()[0]
    values_to_add['city'] = udf['city'].value_counts().keys()[0]
    values_to_add['state'] = udf['state'].value_counts().keys()[0]
    values_to_add['zip'] = udf['zip'].value_counts().keys()[0]
    values_to_add['lat'] = udf['lat'].value_counts().keys()[0]
    values_to_add['long'] = udf['long'].value_counts().keys()[0]
    values_to_add['city_pop'] = udf['city_pop'].value_counts().keys()[0]
    values_to_add['job'] = udf['job'].value_counts().keys()[0]
    values_to_add['age'] = 2022-int(udf['dob'].value_counts().keys()[0][:4])
    row_to_add = pd.Series(values_to_add, name=i)
    usersDF = usersDF.append(row_to_add)
usersDF.to_csv('/content/drive/MyDrive/users_features.csv')

In [ ]:
def get_type(buckets,amt):
    if amt<=buckets['Q1']:
        return "small"
    elif amt<=buckets['Q2']:
        return "medium"
    elif amt<=buckets['Q3']:
        return "large"
    else:
        return "huge"
        
buckets = {'Q1':df1['amt'].quantile(.25), 'Q2':df1['amt'].quantile(0.5),'Q3':df1['amt'].quantile(0.75)}
buckets

{'Q1': 9.69, 'Q2': 46.47, 'Q3': 82.63}

In [ ]:
columns = ['cc_num','name','type','time']
edgesDF = pd.DataFrame(columns=columns)
i = 0
for index,row in tqdm(df1.iterrows(),total=df1.shape[0]):
    values_to_add = {}
    values_to_add['cc_num'] = row['cc_num']
    values_to_add['name'] = row['merchant']
    values_to_add['type'] = get_type(buckets,row['amt'])
    values_to_add['time'] = int(row['trans_date_trans_time'][:4])
    row_to_add = pd.Series(values_to_add,name=i)
    i+=1
    edgesDF = edgesDF.append(row_to_add)
edgesDF.to_csv('/content/drive/MyDrive/edgelist.csv')

100%|██████████| 94876/94876 [10:18<00:00, 153.49it/s]
